## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [3]:
path = "/Users/ecem/Desktop/gyrocardiogram/s-vs-r/"


In [4]:
diseased_s = np.load(path + "data/diseased-10sec-s.npy", allow_pickle= True)
diseased_r = np.load(path + "data/diseased-10sec-r.npy", allow_pickle= True)


In [5]:
print(diseased_s.shape)
print(diseased_r.shape)

(3, 1070, 2560)
(3, 2017, 2560)


## Zero Crossing Rate:
Compute the zero-crossing rate of an audio time series.
The zero-crossing rate (ZCR) is the rate at which a signal changes from positive to zero to negative or from negative to zero to positive.

In [6]:
import librosa as lbs

In [7]:
def zero_crossing(array):
    zero_crossing_rate = np.ones(shape = (array.shape[0],array.shape[1], 7))
    
    for i in range(array.shape[1]):
        zero_crossing_rate[0][i][0:6] = lbs.feature.zero_crossing_rate(np.array(array[0][i]))
        zero_crossing_rate[1][i][0:6] = lbs.feature.zero_crossing_rate(np.array(array[1][i]))
        zero_crossing_rate[2][i][0:6] = lbs.feature.zero_crossing_rate(np.array(array[2][i]))

        zero_crossing_rate[0][i][6] = sum(lbs.zero_crossings(np.array(array[0][i])))
        zero_crossing_rate[1][i][6] = sum(lbs.zero_crossings(np.array(array[1][i])))
        zero_crossing_rate[2][i][6] = sum(lbs.zero_crossings(np.array(array[2][i])))
        
    return zero_crossing_rate

        

In [8]:
zcr_diseased_s = zero_crossing(diseased_s)

In [9]:
zcr_diseased_r = zero_crossing(diseased_r)

In [10]:
print(zcr_diseased_s.shape)
print(zcr_diseased_r.shape)

(3, 1070, 7)
(3, 2017, 7)


In [11]:
columns_zcr = []
for i in range(zcr_diseased_s.shape[2]-1):
    columns_zcr.append("zcr_x{}".format(i))
    columns_zcr.append("zcr_y{}".format(i))
    columns_zcr.append("zcr_z{}".format(i))
    
columns_zcr.append("#ofcrossing_x")
columns_zcr.append("#ofcrossing_y")
columns_zcr.append("#ofcrossing_z")

print(columns_zcr)

['zcr_x0', 'zcr_y0', 'zcr_z0', 'zcr_x1', 'zcr_y1', 'zcr_z1', 'zcr_x2', 'zcr_y2', 'zcr_z2', 'zcr_x3', 'zcr_y3', 'zcr_z3', 'zcr_x4', 'zcr_y4', 'zcr_z4', 'zcr_x5', 'zcr_y5', 'zcr_z5', '#ofcrossing_x', '#ofcrossing_y', '#ofcrossing_z']


In [12]:
def dataframe_creator(array, df = pd.DataFrame(), l = 7, columns = None):
    values = []

    for i in range(array.shape[1]):
        llist = list(array[0][i][0:l-1].tolist() + array[1][i][0:l-1].tolist() 
                        + array [2][i][0:l-1].tolist())
        llist.append(array[0][i][l-1])
        llist.append(array[1][i][l-1])
        llist.append(array[2][i][l-1])

        values.append(llist)
        medium = pd.DataFrame([values[i]], columns = columns)
        
        
        df = pd.concat([df, medium], axis = 0)
    return df 

In [13]:
df = dataframe_creator(zcr_diseased_s, columns = columns_zcr)
df = dataframe_creator(zcr_diseased_r, df = df, columns = columns_zcr )

df.reset_index(drop = True, inplace = True)
df

,zcr_x0,zcr_y0,zcr_z0,zcr_x1,zcr_y1,zcr_z1,zcr_x2,zcr_y2,zcr_z2,zcr_x3,...,zcr_z3,zcr_x4,zcr_y4,zcr_z4,zcr_x5,zcr_y5,zcr_z5,#ofcrossing_x,#ofcrossing_y,#ofcrossing_z
0,0.076172,0.110352,0.122559,0.137207,0.092773,0.058594,0.055664,0.108398,0.145508,0.146973,...,0.063965,0.101074,0.132324,0.202637,0.155273,0.128906,0.097656,347.0,354.0,472.0
1,0.069336,0.098145,0.148438,0.115723,0.099121,0.070312,0.104980,0.142090,0.190430,0.165039,...,0.085938,0.085938,0.123047,0.155273,0.161621,0.118164,0.081055,346.0,468.0,419.0
2,0.080566,0.116699,0.174316,0.156250,0.124512,0.087891,0.078125,0.121094,0.170898,0.165527,...,0.074219,0.080078,0.116211,0.146484,0.167969,0.125977,0.089844,421.0,401.0,423.0
3,0.083984,0.127930,0.168945,0.126465,0.098145,0.054199,0.075684,0.102051,0.145508,0.135254,...,0.068848,0.090820,0.134766,0.178711,0.194824,0.141113,0.097168,374.0,351.0,476.0
4,0.081543,0.129883,0.164551,0.137695,0.119629,0.071289,0.099609,0.137207,0.170898,0.145996,...,0.070801,0.077148,0.118164,0.163086,0.177246,0.128906,0.087891,413.0,427.0,423.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082,0.020996,0.030273,0.047363,0.050781,0.042969,0.033691,0.057617,0.068848,0.095703,0.102051,...,0.047852,0.015137,0.026367,0.036133,0.035156,0.029297,0.018555,132.0,240.0,93.0
3083,0.021973,0.063965,0.074219,0.086426,0.074219,0.031738,0.048340,0.104004,0.144043,0.164062,...,0.078125,0.009766,0.016602,0.030273,0.052734,0.050781,0.043945,198.0,374.0,125.0
3084,0.025391,0.039551,0.046387,0.040527,0.028809,0.014648,0.028809,0.048828,0.057617,0.055664,...,0.020996,0.023438,0.037598,0.044922,0.039551,0.028320,0.014160,112.0,144.0,107.0
3085,0.029297,0.037598,0.050293,0.053223,0.035645,0.027344,0.029297,0.042969,0.058105,0.059082,...,0.035156,0.021973,0.035645,0.041992,0.045410,0.037109,0.023438,134.0,161.0,122.0


In [14]:
df.to_csv(path + 'feature_extr/s-vs-r-zcrs.csv')

### Chromogram:
We can use Chroma feature visualization to know how dominant the characteristics of a certain pitch {C, C♯, D, D♯, E, F, F♯, G, G♯, A, A♯, B} is present in the sampled frame.

Compute a chromagram from a waveform or power spectrogram.
It returns Normalized energy for each chroma bin at each frame.

Hop size should refer to the number of samples in between successive frames. For signal analysis Hop Size should be less than the frame size, so that frames overlap.

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
import librosa.display

In [17]:
lbs.feature.chroma_stft(np.array(diseased_s[0][i]), sr= 256, hop_length = 512)[1].mean()

0.565785391562203

In [18]:
def chromagram_average(array):
    pitch = np.ones(shape = (array.shape[0],array.shape[1], 12)) 
    
    for i in range(array.shape[1]):
        for j in range(12):
            pitch[0][i][j] = lbs.feature.chroma_stft(np.array(array[0][i]), sr= 256, hop_length = 512)[j].mean()
            pitch[1][i][j] = lbs.feature.chroma_stft(np.array(array[1][i]), sr= 256, hop_length = 512)[j].mean()
            pitch[2][i][j] = lbs.feature.chroma_stft(np.array(array[2][i]), sr= 256, hop_length = 512)[j].mean()
        
    return pitch

In [19]:
chro_diseased_s = chromagram_average(diseased_s)
chro_diseased_r = chromagram_average(diseased_r)

In [20]:
print(chro_diseased_s.shape)
print(chro_diseased_r.shape)

(3, 1070, 12)
(3, 2017, 12)


In [21]:
lst = ["C", "C♯", "D", "D♯", "E", "F", "F♯", "G", "G♯", "A", "A♯", "B"]

In [22]:
col_chro = []
for i in range(chro_diseased_s.shape[2]):
    col_chro.append(lst[i]+ "_x")
    col_chro.append(lst[i]+ "_y")
    col_chro.append(lst[i]+ "_z")
    
print(col_chro)
print(len(col_chro))

['C_x', 'C_y', 'C_z', 'C♯_x', 'C♯_y', 'C♯_z', 'D_x', 'D_y', 'D_z', 'D♯_x', 'D♯_y', 'D♯_z', 'E_x', 'E_y', 'E_z', 'F_x', 'F_y', 'F_z', 'F♯_x', 'F♯_y', 'F♯_z', 'G_x', 'G_y', 'G_z', 'G♯_x', 'G♯_y', 'G♯_z', 'A_x', 'A_y', 'A_z', 'A♯_x', 'A♯_y', 'A♯_z', 'B_x', 'B_y', 'B_z']
36


In [23]:
ch_df = dataframe_creator(chro_diseased_s,
                         df =pd.DataFrame(columns = col_chro), columns = col_chro, l = 12)
ch_df = dataframe_creator(chro_diseased_r,
                         df =ch_df, columns = col_chro, l = 12)

ch_df.reset_index(drop = True, inplace = True)
ch_df

,C_x,C_y,C_z,C♯_x,C♯_y,C♯_z,D_x,D_y,D_z,D♯_x,...,G♯_z,A_x,A_y,A_z,A♯_x,A♯_y,A♯_z,B_x,B_y,B_z
0,0.555127,0.551007,0.591269,0.632124,0.716271,0.706916,0.804017,0.739751,0.739839,0.858274,...,0.447238,0.586633,0.586094,0.61845,0.682704,0.806179,0.944554,0.90622,0.816215,0.952385
1,0.650852,0.580121,0.656266,0.771298,0.817694,0.850729,0.795392,0.737869,0.71377,0.801904,...,0.370744,0.54079,0.4942,0.56261,0.686644,0.763773,0.901078,0.966561,0.968191,0.97543
2,0.655348,0.561381,0.513871,0.644764,0.844095,0.755911,0.807829,0.803644,0.691069,0.752769,...,0.471826,0.545105,0.622657,0.719751,0.743706,0.853023,0.963259,0.989587,0.903504,0.977712
3,0.646997,0.571121,0.653658,0.799112,0.770458,0.778966,0.880007,0.848072,0.743724,0.80714,...,0.457498,0.527787,0.621815,0.576349,0.684778,0.908337,0.885729,0.971279,0.85945,0.950192
4,0.66509,0.508255,0.566985,0.676822,0.725616,0.7619,0.779465,0.715341,0.737249,0.792293,...,0.430264,0.504946,0.516594,0.567081,0.651188,0.797746,0.872354,0.993327,0.871348,0.954366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082,0.504617,0.377322,0.4198,0.483778,0.511083,0.568662,0.645658,0.693711,0.697447,0.819131,...,0.441063,0.45886,0.487918,0.585393,0.730276,0.821506,0.84623,0.856822,0.882903,0.975521
3083,0.504839,0.400153,0.540687,0.570051,0.706773,0.755205,0.685645,0.548825,0.547894,0.725591,...,0.634541,0.604767,0.546858,0.877516,0.897308,0.816422,0.764538,0.760204,0.712461,0.872031
3084,0.79041,0.559791,0.575885,0.615956,0.896705,0.872733,0.695856,0.613135,0.626483,0.59136,...,0.782495,0.77971,0.721939,0.802865,0.746536,0.644389,0.67067,0.711055,0.720563,0.645341
3085,0.609852,0.629701,0.576446,0.554601,0.615104,0.782655,0.834541,0.865395,0.783972,0.688445,...,0.600384,0.683958,0.858378,0.812751,0.722344,0.862989,0.785688,0.683079,0.882828,0.656392


In [24]:
ch_df.to_csv(path + "feature_extr/s-vs-r-chromogram.csv")

## Tempogram

Compute the tempogram: local autocorrelation of the onset strength envelope.

# kendime not tempogram değerlerini spektogram değerleri gibi kaydet.

In [25]:
def tempo_average(array): # 384, 11
    means= []
    for i in range(array.shape[0]): #384
        means.append(array[i][0:11].mean())
    return np.array(means) # 384
        

In [26]:
def tempogram(array = 0, hop_lenght = 256):
    hop_length = 256
    sr =256
    oenv = librosa.onset.onset_strength(y=np.array(array), 
                                            sr=sr, 
                                            hop_length=hop_length)

    times = librosa.times_like(oenv, sr=sr, hop_length=hop_length)

    tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr= sr,
                                              hop_length=hop_length)
    # Estimate the global tempo for display purposes
    tempo = librosa.beat.tempo(onset_envelope=oenv, sr= sr,
                                   hop_length=hop_length)[0]
    return [tempogram, tempo]

In [27]:
tempogram(diseased_s[0][0])[0].shape

(384, 11)

In [28]:
tempogram(diseased_s[0][0])[0][12]

array([0.58206686, 0.58440331, 0.58674206, 0.58908321, 0.5914269 ,
       0.59377326, 0.59612239, 0.59847443, 0.60082948, 0.60318768,
       0.60554913])

In [29]:
diseased_s[0][0].shape

(2560,)

In [30]:
def temp_values(array):
    final = np.ones(shape = (3, array.shape[1], 384))
    for i in range(array.shape[1]): #143
        
        _x = tempogram(array[0][i])[0] #1,2560
        _y = tempogram(array[1][i])[0]
        _z = tempogram(array[2][i])[0]

        final[0][i] = tempo_average(_x) #384
        final[1][i] = tempo_average(_y)
        final[2][i] = tempo_average(_z)
        
        
    return final

In [31]:
temp_s = temp_values(diseased_s)
temp_r = temp_values(diseased_r)


In [32]:
print(temp_s.shape)
print(temp_r.shape)

(3, 1070, 384)
(3, 2017, 384)


In [33]:
columns_t = []
for i in range(temp_s.shape[-1]):
    columns_t.append("tempogram_x{}".format(i))
    columns_t.append("tempogram_y{}".format(i))
    columns_t.append("tempogram_z{}".format(i))
print(columns_t)

['tempogram_x0', 'tempogram_y0', 'tempogram_z0', 'tempogram_x1', 'tempogram_y1', 'tempogram_z1', 'tempogram_x2', 'tempogram_y2', 'tempogram_z2', 'tempogram_x3', 'tempogram_y3', 'tempogram_z3', 'tempogram_x4', 'tempogram_y4', 'tempogram_z4', 'tempogram_x5', 'tempogram_y5', 'tempogram_z5', 'tempogram_x6', 'tempogram_y6', 'tempogram_z6', 'tempogram_x7', 'tempogram_y7', 'tempogram_z7', 'tempogram_x8', 'tempogram_y8', 'tempogram_z8', 'tempogram_x9', 'tempogram_y9', 'tempogram_z9', 'tempogram_x10', 'tempogram_y10', 'tempogram_z10', 'tempogram_x11', 'tempogram_y11', 'tempogram_z11', 'tempogram_x12', 'tempogram_y12', 'tempogram_z12', 'tempogram_x13', 'tempogram_y13', 'tempogram_z13', 'tempogram_x14', 'tempogram_y14', 'tempogram_z14', 'tempogram_x15', 'tempogram_y15', 'tempogram_z15', 'tempogram_x16', 'tempogram_y16', 'tempogram_z16', 'tempogram_x17', 'tempogram_y17', 'tempogram_z17', 'tempogram_x18', 'tempogram_y18', 'tempogram_z18', 'tempogram_x19', 'tempogram_y19', 'tempogram_z19', 'tempogra

In [34]:
len(columns_t)

1152

In [35]:
df_t = dataframe_creator(temp_s,
                         df =pd.DataFrame(columns = columns_t), columns = columns_t, l = 384)
df_t = dataframe_creator(temp_r,
                         df = df_t, columns = columns_t, l = 384)

df_t.reset_index(drop = True, inplace = True)
df_t

,tempogram_x0,tempogram_y0,tempogram_z0,tempogram_x1,tempogram_y1,tempogram_z1,tempogram_x2,tempogram_y2,tempogram_z2,tempogram_x3,...,tempogram_z380,tempogram_x381,tempogram_y381,tempogram_z381,tempogram_x382,tempogram_y382,tempogram_z382,tempogram_x383,tempogram_y383,tempogram_z383
0,1.0,0.773956,0.692283,0.657054,0.650099,0.644578,0.637382,0.630162,0.622921,0.615661,...,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.614576,0.489529,0.471896,0.475368,0.465189,0.460128,0.455048,0.44995,0.444837,...,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.699706,0.59222,0.557152,0.533122,0.523759,0.518049,0.512319,0.506569,0.5008,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,0.0
3,1.0,0.740545,0.639777,0.619184,0.614569,0.610662,0.603863,0.597042,0.5902,0.58334,...,-0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0
4,1.0,0.783244,0.698631,0.671796,0.662328,0.653183,0.645886,0.638566,0.631223,0.623862,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082,1.0,0.728353,0.639177,0.612537,0.580257,0.549647,0.543659,0.537647,0.531615,0.525564,...,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3083,1.0,0.992773,0.983609,0.975166,0.969481,0.959322,0.947528,0.935717,0.923893,0.912059,...,-0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3084,1.0,0.00518,0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,...,0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.0
3085,1.0,0.931978,0.897683,0.91115,0.923446,0.90549,0.894788,0.884064,0.87332,0.862559,...,-0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0


In [36]:
df_t.to_csv(path + 'feature_extr/s-vs-r-tempogram.csv')

In [37]:
df_t.isin([0]).sum(axis=0)

tempogram_x0      0
tempogram_y0      0
tempogram_z0      0
tempogram_x1      0
tempogram_y1      0
                 ..
tempogram_y382    0
tempogram_z382    0
tempogram_x383    0
tempogram_y383    0
tempogram_z383    0
Length: 1152, dtype: int64

In [38]:
df_t['tempogram_x382'][10]

4.731797856249001e-17

In [39]:
final_df = pd.concat([df, ch_df], axis =1)
final_df

,zcr_x0,zcr_y0,zcr_z0,zcr_x1,zcr_y1,zcr_z1,zcr_x2,zcr_y2,zcr_z2,zcr_x3,...,G♯_z,A_x,A_y,A_z,A♯_x,A♯_y,A♯_z,B_x,B_y,B_z
0,0.076172,0.110352,0.122559,0.137207,0.092773,0.058594,0.055664,0.108398,0.145508,0.146973,...,0.447238,0.586633,0.586094,0.61845,0.682704,0.806179,0.944554,0.90622,0.816215,0.952385
1,0.069336,0.098145,0.148438,0.115723,0.099121,0.070312,0.104980,0.142090,0.190430,0.165039,...,0.370744,0.54079,0.4942,0.56261,0.686644,0.763773,0.901078,0.966561,0.968191,0.97543
2,0.080566,0.116699,0.174316,0.156250,0.124512,0.087891,0.078125,0.121094,0.170898,0.165527,...,0.471826,0.545105,0.622657,0.719751,0.743706,0.853023,0.963259,0.989587,0.903504,0.977712
3,0.083984,0.127930,0.168945,0.126465,0.098145,0.054199,0.075684,0.102051,0.145508,0.135254,...,0.457498,0.527787,0.621815,0.576349,0.684778,0.908337,0.885729,0.971279,0.85945,0.950192
4,0.081543,0.129883,0.164551,0.137695,0.119629,0.071289,0.099609,0.137207,0.170898,0.145996,...,0.430264,0.504946,0.516594,0.567081,0.651188,0.797746,0.872354,0.993327,0.871348,0.954366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082,0.020996,0.030273,0.047363,0.050781,0.042969,0.033691,0.057617,0.068848,0.095703,0.102051,...,0.441063,0.45886,0.487918,0.585393,0.730276,0.821506,0.84623,0.856822,0.882903,0.975521
3083,0.021973,0.063965,0.074219,0.086426,0.074219,0.031738,0.048340,0.104004,0.144043,0.164062,...,0.634541,0.604767,0.546858,0.877516,0.897308,0.816422,0.764538,0.760204,0.712461,0.872031
3084,0.025391,0.039551,0.046387,0.040527,0.028809,0.014648,0.028809,0.048828,0.057617,0.055664,...,0.782495,0.77971,0.721939,0.802865,0.746536,0.644389,0.67067,0.711055,0.720563,0.645341
3085,0.029297,0.037598,0.050293,0.053223,0.035645,0.027344,0.029297,0.042969,0.058105,0.059082,...,0.600384,0.683958,0.858378,0.812751,0.722344,0.862989,0.785688,0.683079,0.882828,0.656392


In [40]:
final_df = pd.concat([final_df, df_t], axis=1)
final_df

,zcr_x0,zcr_y0,zcr_z0,zcr_x1,zcr_y1,zcr_z1,zcr_x2,zcr_y2,zcr_z2,zcr_x3,...,tempogram_z380,tempogram_x381,tempogram_y381,tempogram_z381,tempogram_x382,tempogram_y382,tempogram_z382,tempogram_x383,tempogram_y383,tempogram_z383
0,0.076172,0.110352,0.122559,0.137207,0.092773,0.058594,0.055664,0.108398,0.145508,0.146973,...,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.069336,0.098145,0.148438,0.115723,0.099121,0.070312,0.104980,0.142090,0.190430,0.165039,...,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.080566,0.116699,0.174316,0.156250,0.124512,0.087891,0.078125,0.121094,0.170898,0.165527,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,0.0
3,0.083984,0.127930,0.168945,0.126465,0.098145,0.054199,0.075684,0.102051,0.145508,0.135254,...,-0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0
4,0.081543,0.129883,0.164551,0.137695,0.119629,0.071289,0.099609,0.137207,0.170898,0.145996,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082,0.020996,0.030273,0.047363,0.050781,0.042969,0.033691,0.057617,0.068848,0.095703,0.102051,...,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3083,0.021973,0.063965,0.074219,0.086426,0.074219,0.031738,0.048340,0.104004,0.144043,0.164062,...,-0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3084,0.025391,0.039551,0.046387,0.040527,0.028809,0.014648,0.028809,0.048828,0.057617,0.055664,...,0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.0
3085,0.029297,0.037598,0.050293,0.053223,0.035645,0.027344,0.029297,0.042969,0.058105,0.059082,...,-0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0


In [41]:
final_df.to_csv(path + 'feature_extr/s-vs-r-final-signal_processing.csv')